# Factor Analysis using the CAPM and Fama-French Factor models
The main idea in Factor Analysis is to take a set of observed returns and decompose it into a set of explanatory returns. We'll follow _Asset Management_ (Ang 2014, Oxford University Press) Chapter 10 and analyze the returns of Berkshire Hathaway. First, we'll need the returns of Berkshire Hathaway which are contained in `brka_d_ret.csv`.

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import edhec_risk_kit as erk
import statsmodels.api as sm

In [2]:
brka_d = pd.read_csv('brka_d_ret.csv', parse_dates=True, index_col=0)
brka_d.head()

,BRKA
DATE,
1990-01-02,-0.005764
1990-01-03,0.000000
1990-01-04,0.005797
1990-01-05,-0.005764
1990-01-08,0.000000


In [3]:
brka_d.tail()

,BRKA
DATE,
2018-12-24,-0.018611
2018-12-26,0.043200
2018-12-27,0.012379
2018-12-28,0.013735
2018-12-31,0.011236


Next, convert these to monthly returns. We want to compound the returns, and we have the `compound` function in the toolkit, so let's load that up and then apply to the monthly returns.

In [4]:
brka_m = brka_d.resample('M').apply(erk.compound).to_period('M')
brka_m.head()

,BRKA
DATE,
1990-01,-0.140634
1990-02,-0.030852
1990-03,-0.069204
1990-04,-0.003717
1990-05,0.067164


In [5]:
brka_m.to_csv('brka_m.csv')

Next, we need to load the explanatory variables, which is the Fama-French monthly returns dataset.

In [6]:
fff = erk.get_fff_returns()
fff.head()

,Mkt-RF,SMB,HML,RF
1926-07,0.0296,-0.0230,-0.0287,0.0022
1926-08,0.0264,-0.0140,0.0419,0.0025
1926-09,0.0036,-0.0132,0.0001,0.0023
1926-10,-0.0324,0.0004,0.0051,0.0032
1926-11,0.0253,-0.0020,-0.0035,0.0031


Next, we decompose the observed BRKA 1990-May 2012 as in Ang (2014) into the portion that's due to the market and the rest that is not due to the market using the CAPM as the explanatory model. Run a linear regression.

In [7]:
brka_excess = brka_m['1990':'2012-05'] - fff.loc['1990':'2012-05', ['RF']].values
mkt_excess = fff.loc['1990':'2012-05', ['Mkt-RF']]
exp_var = mkt_excess.copy()
exp_var['Constant'] = 1
lm = sm.OLS(brka_excess, exp_var).fit()

In [8]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   BRKA   R-squared:                       0.154
Model:                            OLS   Adj. R-squared:                  0.150
Method:                 Least Squares   F-statistic:                     48.45
Date:                Wed, 27 May 2020   Prob (F-statistic):           2.62e-11
Time:                        21:32:14   Log-Likelihood:                 388.47
No. Observations:                 269   AIC:                            -772.9
Df Residuals:                     267   BIC:                            -765.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         0.5402      0.078      6.961      0.000       0.387       0.693
Constant       0.0061      0.004      1.744      0.082      -0.001       0.013
==============================================================================
Omnibus:                       45.698   Durbin-Watson:                   2.079
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              102.573
Skew:                           0.825   Prob(JB):                     5.33e-23
Kurtosis:                       5.535   Cond. No.                         22.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## CAPM benchmark interpretation
This implies that the CAPM benchmark consists of 46 cents in T-Bills and 54 cents in the market. Relative to this, Berkshire Hathaway is adding (has an alpha of) 0.61% (per month) although the degree of statistical significance isn't very high.

Next, let's add some additional explanatory variables; size and value.

In [9]:
exp_var['Value'] = fff.loc['1990':'2012-05', ['HML']]
exp_var['Size'] = fff.loc['1990': '2012-05', ['SMB']]
exp_var.head()

,Mkt-RF,Constant,Value,Size
1990-01,-0.0785,1,0.0087,-0.0129
1990-02,0.0111,1,0.0061,0.0103
1990-03,0.0183,1,-0.0290,0.0152
1990-04,-0.0336,1,-0.0255,-0.0050
1990-05,0.0842,1,-0.0374,-0.0257


In [10]:
lm = sm.OLS(brka_excess, exp_var).fit()
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   BRKA   R-squared:                       0.290
Model:                            OLS   Adj. R-squared:                  0.282
Method:                 Least Squares   F-statistic:                     36.06
Date:                Wed, 27 May 2020   Prob (F-statistic):           1.41e-19
Time:                        21:32:14   Log-Likelihood:                 412.09
No. Observations:                 269   AIC:                            -816.2
Df Residuals:                     265   BIC:                            -801.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         0.6761      0.074      9.155      0.000       0.531       0.821
Constant       0.0055      0.003      1.679      0.094      -0.001       0.012
Value          0.3814      0.109      3.508      0.001       0.167       0.595
Size          -0.5023      0.101     -4.962      0.000      -0.702      -0.303
==============================================================================
Omnibus:                       42.261   Durbin-Watson:                   2.146
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               67.954
Skew:                           0.904   Prob(JB):                     1.75e-15
Kurtosis:                       4.671   Cond. No.                         37.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## The Fama-French Benchmark Interpretation
Alpha has fallen from 0.61% to 0.55% per month. Loading on the market has moved up from 0.54 to 0.67 which means the new explanatory variables changed things; irrelevant variables would've caused the loading on the market to remain unaffected.

Loading on Value being positive suggests that Hathaway has a significant value tilt, and loading on Size being negative suggests that Hathaway tends to invest in large companies not small ones.

In [15]:
result = erk.regress(brka_excess, mkt_excess)

In [16]:
result.params

Mkt-RF    0.540175
Alpha     0.006133
dtype: float64